In [23]:
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import os
import certifi
import pandas as pd
import requests

from functools import reduce
from datetime import date
from fake_useragent import UserAgent
from IPython.display import clear_output

In [24]:
# csv파일을 String(object)으로 읽음
df = pd.read_csv("BRNO_250730.csv", dtype = object)
filteredDf = pd.DataFrame()
filteredDf['b_no'] = df['BIZ_REGI_NO']
print(f'전체 검사 대상 사업자번호 수는 {len(filteredDf)} 입니다')

# 개인/법인 구분코드(2자리)
# - 개인구분코드
# (1) 개인과세사업자는 특정 동 구별 없이 01~79까지를 순차적으로 부여
# (2) 개인면세사업자는 산업 구분없이 90~99까지를 순차적으로 부여
# (3) 소득세법 제2조 제3항에 해당하는 법인이 아닌 종교 단체 : 89
# (4) 소득세법 제2조 제3앙에 핻아하는 자로서 (3)이외의자(아파트관리사무소 등) 및 다단계판매원 : 80
# - 법인성격코드
# (1) 영리법인의 본점 : 81, 86, 87, 88
# (2) 비영리법인의 본점 및 지점(법인격 없는 사단, 재단, 기타 단체 중 법인으로 보는 단체를 포함) : 82
# (3) 국가, 지방자치단체, 지방자치단체조합 : 83
# (4) 외국법인의 본/지점 및 연락사무소 : 84
# (5) 영리법인의 지점 : 85

# 일련번호코드(4자리)
# - 과세사업자(일반과세자, 간이과세자), 면세사업자, 법인사업자별로 등록 또는 지정 일자순으로 사용가능한 번호를 0001~9999로 부여

# 검증번호(1자리)
# 전산시스템에 의하여 사업자등록번호의 오류여부를 검증하기 위하여 1자리의 검증번호를 부여
filteredDf = filteredDf[filteredDf.b_no.str[0] != '0']
filteredDf = filteredDf.reset_index(drop=True)
print(f'사업자번호가 0으로 시작하는 기업을 제외한 검사 대상 사업자번호 수는 {len(filteredDf)} 입니다')

전체 검사 대상 사업자번호 수는 494643 입니다
사업자번호가 0으로 시작하는 기업을 제외한 검사 대상 사업자번호 수는 494643 입니다


In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = 'https://api.odcloud.kr/api/nts-businessman/v1/status'

headers = {
    'Content-Type': 'application/json',  # JSON 전송 시 꼭 지정
    'Accept': 'application/json',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

params = {
    'serviceKey': '6No4JLBZPafGmvs0YQ/WiHSNqVrvqqAox0frrHQJbt/vc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q==',
}

# URL요청 함수
# API서버가 request를 로봇으로 인식하면 connection을 차단하므로, 이를 예외처리함(User-Agent 정보를 변경하여 재귀호출)
def urlRequest(url, headers, params, payload):
    try:
        response = requests.post(url, headers=headers, params=params, json=payload, verify=False)
        return response
    except:
        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
            "User-Agent": UserAgent().random
        }
        return urlRequest(headers)
        
dfReturnValue = pd.DataFrame()
for n in range(0, len(filteredDf), 100):
    clear_output(wait=True) # output cell을 clear함
    print(f'진행중: {n} 번째')
    endPoint = min(n + 100, len(filteredDf) + 1)
    dfBRNO = filteredDf.iloc[n:endPoint, :] # 사업자번호를 100개 단위로 자름
    payload = dfBRNO.to_dict('list') # 잘라낸 사업자번호 100개를 dictionary 형태로 변경
    response = urlRequest(url, headers, params, payload)
    jsResponse = response.json()['data']
    dfReturnValue = pd.concat([dfReturnValue, pd.DataFrame(jsResponse)], axis=0, ignore_index = True)
    display(dfReturnValue) # 현재 진행상태를 파악하기 위한 출력
dfApiCallResult = dfReturnValue # b_stt_cd (1:정상, 2:휴업, 3:폐업)

# 크롤링 날짜를 기준으로 기준년월(GG_YM)값을 삽입하고 그 결과를 csv로 내보냄
today = date.today()
ggYm = today.strftime("%Y") + today.strftime("%m")
dfApiCallResult.insert(0, column='GG_YM', value=ggYm)
dfApiCallResult.to_csv(f"IT_EFAS_BIZ_STT_{today}.csv", index=False) # 인덱스를 지우고 csv로 저장

진행중: 130300 번째


,b_no,b_stt,b_stt_cd,tax_type,tax_type_cd,end_dt,utcc_yn,tax_type_change_dt,invoice_apply_dt,rbf_tax_type,rbf_tax_type_cd
0,1010112922,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
1,1010114615,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
2,1010116082,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
3,1010118644,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
4,1010124729,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
...,...,...,...,...,...,...,...,...,...,...,...
130395,2150285473,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
130396,2150290428,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
130397,2150290715,계속사업자,01,부가가치세 일반과세자,01,,N,,,해당없음,99
130398,2150321229,계속사업자,01,부가가치세 일반과세자,01,,N,19980701,,부가가치세 간이과세자,02
